In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [188]:
# export
from exp.nb_00 import *
from exp.nb_01 import *
import os
import shutil
import re
from climatools.lblnew.export import vector_to_F77

In [122]:
param = LBLnewBestfitSWParam(band=9, vmin=8200, vmax=14290, nv=10000, dv=0.001,
                     molecule='h2o', conc='atmpro',
                     ref_pts=[(300, 250), (300, 250)], ng_refs=[4, 6], ng_adju=[-3, 0],
                     wgt=[(.95, .9, .5, .5), (.5, .5, .5, .7, .85, .95)],
                     cosz=1., rsfc=0,
                     klin=3e-25,
                     option_k_lookup=0)

In [123]:
param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

## Creating work directory and copy source code into it

In [12]:
PATH = Path('test_run')

In [13]:
PATH.mkdir(exist_ok=True)

In [14]:
srcfiles = [SRC/n for n in FNAMES]

In [21]:
srcfiles

[PosixPath('/chia_cluster/home/jackyu/radiation/crdnew-sw/lblnew-bestfit-sw.f'),
 PosixPath('/chia_cluster/home/jackyu/radiation/crdnew-sw/lblcom.f')]

In [556]:
for n in srcfiles: shutil.copy(n, PATH)

In [16]:
list(PATH.iterdir())

[PosixPath('test_run/lblnew-bestfit-sw.f'),
 PosixPath('test_run/lblcom.f'),
 PosixPath('test_run/lblnew-bestfit-sw-replace.f')]

## Reading the source code for the main program  
We are going to insert the input parameter values into this

In [557]:
S = open(PATH/'lblnew-bestfit-sw.f', mode='r').read()
#with open(PATH/'lblnew-bestfit-sw-replace.f', mode='w', encoding='utf-8') as f: f.write(s)

In [107]:
#! cp {PATH/'lblnew-bestfit-sw-replace.f'} /chia_cluster/home/jackyu/radiation/crdnew-sw/.

## Writing values into source code

In [460]:
# export
def pat_parameter(name):                                                                                        
    '''                                                                                                         
    Returns regular expression for assigning value to                                                           
    a parameter variable in Fortran.                                                                            
    '''                                                                                                         
    return r'''(\n [^!\n]+ parameter .* :: \s* &? \s* {name} \s* = ) (.*)'''.format(name=name) 

def enter_parameters(s, **kwargs):
    '''
    Writes parameter values to the source code.
    '''
    for n, vs in kwargs.items():
        regex = re.compile(pat_parameter(n), re.VERBOSE)
        try: v = '(/' + ', '.join([str(v) for v in vs]) + '/)'
        except TypeError: v = str(vs)
        s = regex.sub(r'\g<1> ' + v, s)
    return s

In [461]:
#export
def pat_data(name):
    dataname = ',\s+'.join(name.split(','))
    return r'''
    (\n \s+ data \s+ (?:{dataname}))
    ([^/,]+ / [^/]+ /)
    '''.format(dataname=dataname)

def enter_data(s, dtype=float, **kwargs):
    '''
    Write `data` values to the source code.
    '''
    for n, v in kwargs.items():
        regex = re.compile(pat_data(n), re.VERBOSE)
        v = vector_to_F77(v, dtype=dtype)
        v = ('\n' + 5 * ' ' + '&/' + '\n') + v + ('\n' + 5 * ' ' + '&/')
        s = regex.sub(r'\g<1>' + v, s)
    return s

#### Spectral parameters

In [ ]:
# Spectral parameters
['dv', 'nv', 'vmin', 'vmax', ], 
["real, parameter :: dv = 0.001",
 "integer, parameter :: nv = 10000",
 "real, parameter :: vstar = 8200.0",
 "integer, parameter :: nband = 609"]

In [465]:
# export 
def spectral_parameters(vmin=None, vmax=None, dv=None, nv=None):
    '''
    Return parameter-value dictionary for the spectral parameters.
    
    Example
    -------
    ```
    pvs = spectral_parameters(vmin=0, vmax=59595959, dv=.5, nv=100)
    print(enter_parameters(S, **pvs))
    ```
    '''
    vstar, nband = vmin, int((vmax - vmin) / (nv * dv))
    return {'vstar':vstar, 'nband':nband, 'nv':nv, 'dv':dv}

#### Absorber parameters

In [ ]:
# Absorber parameters
['molecule', 'conc'], ['''
      integer   flgh2o, flgco2, flgo3, flgo2
      data      flgh2o, flgco2, flgo3, flgo2
     $        /   1,      0,      0,      0 /
 ''',
 '''
        do k=1,nlayer
        if(mid.eq.1) xlayer(k)=wlayer(k)
        if(mid.eq.2) xlayer(k)=clayer(k) 
        if(mid.eq.3) xlayer(k)=olayer(k) 
        if(mid.eq.4) xlayer(k)=0.2315 
       end do
 ''']

In [540]:
# export
def pat_conc(name):
    return r'''
    (\n \s+ {name} \s* = \s* )([-+\.\deE]+)
    '''.format(name=name)

In [541]:
# export
def conc_values(co2=None):
    '''
    Return variable-value dictionary for absorber concentrations.
    Note that this sets all layers' concentration to a constant.
    
    Example
    -------
    ```
    pvs = conc_values(co2=500e-6)
    enter_conc(S, **pvs)
    ```
    '''
    pvs = {}
    if co2: pvs['clayer'] = co2
    return pvs

In [542]:
# export
def enter_conc(s, **kwargs):
    for n, v in kwargs.items():
        regex = re.compile(pat_conc(n), re.VERBOSE)
        s = regex.sub(r'\g<1>' + str(v), s, 1)
    return s    

In [543]:
# export
def molecule_flags_data(molecule=None):
    '''
    Return data-value dictionary for absorber flags.
    
    Example
    -------
    ```
    pvs = molecule_flags_data(molecule='co2')
    enter_data(S, dtype=int, **pvs)
    ```
    '''
    ns = ('h2o', 'co2', 'o3', 'o2')
    if molecule not in ns: raise Exception('Input molecule must be h2o, co2, o3 or o2.')
    n = ', '.join(['flg' + n for n in ns])
    v = np.array([1 if n == molecule else 0 for n in ns])
    return {n:v}    

#### Opitcal parameters

In [146]:
# Optics parameters
['cosz', 'rsfc'], ["parameter (cosz=.2588,rsfc=0.0) "]

(['cosz', 'rsfc'], ['parameter (cosz=.2588,rsfc=0.0) '])

In [551]:
#export
def optical_parameters(cosz=None, rsfc=None):
    '''
    Example
    -------
    ```
    pvs = optical_parameters(cosz=.4556, rsfc=.78)
    print(enter_parameters(S, **pvs))
    ```
    '''
    pvs = {}
    if cosz: pvs['cosz'] = cosz
    if rsfc: pvs['rsfc'] = rsfc
    return pvs

#### Atmosphere profile

In [147]:
# Atmosphere profile
['atmpro'], ['''
     include '/chia_cluster/home/jackyu/radiation/
     &crdnew-sw/atmosphere_profiles/mls75.pro'
 ''']

(['atmpro'],
 ["\n     include '/chia_cluster/home/jackyu/radiation/\n     &crdnew-sw/atmosphere_profiles/mls75.pro'\n "])

In [471]:
#export
def pat_atmpro():
    '''
    Return regular expression that matches the assignment of the 
    atmosphere profilein the source code.
    '''
    return r'''(\n [^!\n]+ atmosphere_profiles/)([a-z]{3,3})'''

In [472]:
#export
def enter_atmpro(s, atmpro=None):
    '''
    Enter atmosphere profile name into the source code.
    
    Example
    -------
    ```
    S = enter_atmpro(S, atmpro='saw')
    ```
    '''
    if atmpro:
        regex = re.compile(pat_atmpro(), re.VERBOSE)
        print(regex.findall(S))
        s = regex.sub(r'\g<1>' + atmpro, s)
    return s

#### k-distribution method parameters

In [144]:
# k-distribution method
['pt_refs'], ["real, dimension(nref), parameter :: p_refs =(/ 300., 300. /)",
              "real, dimension(nref), parameter :: t_refs =(/ 250., 250. /)",
              "integer, parameter :: nref = 2"]
['ng_refs'], ["integer, dimension(nref), parameter :: ng_refs = (/4, 6/)",
              "integer, parameter :: ng = 10"]
["ng_adju"], ["integer, dimension(nref), parameter :: ng_adju = (/ -3, 0 /)"]
["klin"], ["real*8, parameter :: klin = 3.e-25"]
['wgt'], ["data wgt \n &     / 0.95, 0.90, 5*0.50, 0.70, 0.85, 0.95 /"]

(['wgt'], ['data wgt \n &     / 0.95, 0.90, 5*0.50, 0.70, 0.85, 0.95 /'])

In [240]:
# export

def kdist_parameters(ref_pts=None, ng_refs=None, ng_adju=None, klin=None):
    '''
    Return parameter-value dictionary for k-distribution method's parameters.
    
    Example
    -------
    ```
    pvs = kdist_parameters(ref_pts=[(500, 2315634563), (11, 257)], ng_refs=[7, 18], ng_adju=[-4, +7], klin=3.3333e-19)
    S = enter_parameters(S, **pvs)
    ```
    '''
    pvs = {}
    if ref_pts: pvs['p_refs'], pvs['t_refs'] = zip(*ref_pts)    
    if ng_refs: pvs['ng_refs'] = ng_refs
    if ng_adju: pvs['ng_adju'] = ng_adju
    if klin: pvs.update({'option_klin':1, 'klin':klin})
    else: pvs.update({'option_klin':0})
    return pvs

In [456]:
#export
def kdist_data(wgt=None):
    '''
    Return parameter-value dictionary for k-distribution method's data vectors.
    
    Example
    -------
    ```
    pvs = kdist_data(wgt=param.wgt)
    enter_data(S, **pvs)
    ```
    '''
    pvs = {}
    if wgt: pvs['wgt'] = np.array([w for ws in wgt for w in ws])
    return pvs

# fin

In [550]:
! python notebook2script.py 02_runner.ipynb

Converted 02_runner.ipynb to exp/nb_02.py
